# Übung 2 - Polygonzug



In [3]:
# Libraries
import numpy as np
import pandas as pd

In [9]:
#import data
data_Noise = pd.read_csv("./data/data_Noise.txt", delimiter=";")
data_NoNoise = pd.read_csv("./data/data_NoNoise.txt", delimiter=";")
fixpunkte = pd.read_csv("./data/fixpunkte.txt", delimiter=";")
i_meteo = pd.read_csv("./data/i_meteo.txt", delimiter=";")

data_NoNoise

,Visur_von,Visur_nach,RI,RII,ZI,ZII,D
0,A,P,165.5718,365.5718,103.4797,296.5203,NaN
1,A,PP1,365.7370,165.7370,96.7342,303.2658,27.4199
2,PP1,A,261.8157,61.8157,103.2658,296.7342,27.4199
3,PP1,PP2,60.9770,260.9770,97.2136,302.7864,25.4368
4,PP2,PP1,70.4558,270.4558,102.7864,297.2136,25.4368
5,PP2,PP3,251.6220,51.6220,98.3621,301.6379,33.4681
6,PP3,PP2,371.0396,171.0396,101.6379,298.3621,33.4681
7,PP3,PP4,151.6620,351.6620,99.3118,300.6882,23.3132
8,PP4,PP3,340.9986,140.9986,100.6882,299.3118,23.3132
9,PP4,E,168.9636,368.9636,100.4986,299.5014,44.8211
